In [17]:
import hashlib
import math
import os
import array
import pickle
import time
import random
import numpy as np
from Crypto.Hash import keccak

class qTesla:
    
    def __init__(self,n=1024):
        # prime
        self.n = n
        self.isHere = False
        self.sigma = 8.5
        
        self.q = 343576577
        
        # Ring Stuff
        # Defining The Ring Z_q[X]/(X^n + 1)
        self.F = GF(self.q)
        self.R_original = PolynomialRing(self.F,'x')
        self.R_alternate = ZZ["x"]
        self.R_alt_quo = self.R_alternate.quotient(x**(self.n) + 1,"a")
        R = self.R_original.quotient(x**(self.n) + 1,"a")
        self.R = R
        
        w = (self.R_original).gen();
        self.w = w
        
        
        g = (w^n + 1)

        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])
        
        
        # Gauss Sampler
        self.beta = 64
        self.t = 78
        self.c = 512
    
    
        self.Ls = 554
        self.Le = 554
        # number of seeds for e_polys
        self.k = 4
        
        # Number of bits per seed in the XOF
        self.k_bits = 256
        self.k_bytes = self.k_bits/8
        
        # rate_xof
        self.rate_xof = 168
        
        #b-GenA
        self.b_GenA = 108
        
        self.b = None
        
        # Short values
        self.B = (2**19) - 1
        self.S = 554
        self.E = 554
        
        self.d = 22
        self.h = 25
        self.cdt_v = [ 0x00000000, 0x00000000,0x0601F22A, 0x280663D4,0x11F09FFA, 0x162FE23D,0x1DA089E9, 0x437226E8,0x28EAB25D, 0x04C51FE2,0x33AC2F26, 0x14FDBA70,0x3DC767DC, 0x4565C960,0x4724FC62, 0x3342C78A,0x4FB448F4, 0x5229D06D,0x576B8599, 0x7423407F,0x5E4786DA, 0x3210BAF7,0x644B2C92, 0x431B3947,0x697E90CE, 0x77C362C4,0x6DEE0B96, 0x2798C9CE,0x71A92144, 0x5765FCE4,0x74C16FD5, 0x1E2A0990,0x7749AC92, 0x0DF36EEB,0x7954BFA4, 0x28079289,0x7AF5067A, 0x2EDC2050,0x7C3BC17C, 0x123D5E7B,0x7D38AD76, 0x2A9381D9,0x7DF9C5DF, 0x0E868CA7,0x7E8B2ABA, 0x18E5C811,0x7EF7237C, 0x00908272,0x7F4637C5, 0x6DBA5126,0x7F7F5707, 0x4A52EDEB,0x7FA808CC, 0x23290599,0x7FC4A083, 0x69BDF2D5,0x7FD870CA, 0x42275558,0x7FEB5D, 0x3EF82C1B,0x7FEF1BFA, 0x6C03A362,0x7FF52D4E, 0x316C2C8C,0x7FF927BA, 0x12AE54AF,0x7FFBBA43, 0x749CC0E2,0x7FFD5E3D, 0x4524AD91,0x7FFE6664, 0x535785B5,0x7FFF0A41, 0x0B291681,0x7FFF6E81, 0x132C3D6F,0x7FFFAAFE, 0x4DBC6BED,0x7FFFCEFD, 0x7A1E2D14,0x7FFFE41E, 0x4C6EC115,0x7FFFF059, 0x319503C8,0x7FFFF754, 0x5DDD0D40,0x7FFFFB43, 0x0B9E9823,0x7FFFFD71, 0x76B81AE1,0x7FFFFEA3, 0x7E66A1EC,0x7FFFFF49, 0x26F6E191,0x7FFFFFA1, 0x2FA31694,0x7FFFFFCF, 0x5247BEC9,0x7FFFFFE7, 0x4F4127C7,0x7FFFFFF3, 0x6FAA69FD,0x7FFFFFFA, 0x0630D073,0x7FFFFFFD, 0x0F2957BB,0x7FFFFFFE, 0x4FD29432,0x7FFFFFFF, 0x2CFAD60D,0x7FFFFFFF, 0x5967A930,0x7FFFFFFF, 0x6E4C9DFF,0x7FFFFFFF, 0x77FDCCC8,0x7FFFFFFF, 0x7C6CE89E,0x7FFFFFFF, 0x7E6D116F,0x7FFFFFFF, 0x7F50FA31,0x7FFFFFFF, 0x7FB50089,0x7FFFFFFF, 0x7FE04C2D,0x7FFFFFFF, 0x7FF2C7C1,0x7FFFFFFF, 0x7FFA8FE3,0x7FFFFFFF, 0x7FFDCB1B,0x7FFFFFFF, 0x7FFF1DE2,0x7FFFFFFF, 0x7FFFA6B7,0x7FFFFFFF, 0x7FFFDD39,0x7FFFFFFF, 0x7FFFF2A3,0x7FFFFFFF, 0x7FFFFAEF,0x7FFFFFFF, 0x7FFFFE1B,0x7FFFFFFF, 0x7FFFFF4D,0x7FFFFFFF, 0x7FFFFFBF,0x7FFFFFFF, 0x7FFFFFE9,0x7FFFFFFF, 0x7FFFFFF8, 0x7FFFFFFF, 0x7FFFFFFD, 0x7FFFFFFF, 0x7FFFFFFF,
]
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; 
            xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ;
            f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ;
            ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ;
            ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ;
                b = s*ff1[i]  
                ff[i] = a + b ; 
                ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
    
    def ntt_inverse(self,ff):               
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def Enc(self,c_prime):
        D = 0
        cnt = 0
        r_s = self.cShake_128(c_prime,self.rate_xof,D)
        
        pos_list = []
        sign_list = []
        
        i = 0
        c = [0] * self.n
        
        while i < self.h:
            if cnt > (self.rate_xof - 3):
                D = D+1


                cnt = 0
                r_s = self.cShake_128(c_prime,self.rate_xof,D)
                
            pos = (r_s[cnt]*(2**8) + r_s[cnt+1]) % self.n
      
            if c[pos] == 0:
                if (r_s[cnt+2] % 2) == 1:
                    c[pos] = -1
                else:
                    c[pos] = 1
                
                pos_list.append(pos)
                sign_list.append(c[pos])
                i+=1
            cnt+=3
            
        return (pos_list,sign_list)
    
    def GenA(self,seed_a):
        self.b =  math.ceil(math.log(self.q,2)/8)
        D = 0
        b_prime = self.b_GenA
        c_s = self.cShake_128(seed_a,self.rate_xof * b_prime,D)
        i = 0
        pos = 0
        j = 0
        number = 0
        a_polys = [[]]

        while i < (self.k*self.n):
            if pos >= (math.floor((self.rate_xof * b_prime)/ self.b)-1):
                D += 1
                pos = 0
                b_prime = 1
                c_s = self.cShake_128(seed_a,self.rate_xof * b_prime,D)
            
            number = int.from_bytes((c_s[pos*self.b:(pos+1)*self.b]),byteorder="little")

            if (number % (2 ** (math.ceil(math.log(self.q,2))))) < self.q:
                    
                a_polys[j].append(number % (2 ** math.ceil(math.log(self.q,2))))
                i += 1
                
                if (i % self.n) == 0:
                    j += 1
                    a_polys.append([])
                
            pos += 1
        
        return a_polys[:-1]
    
    def GaussSampler(self,seed,counter):
        D_prime = counter * 2**8
        i = 0
        z = [0]*self.n
        
        while i < self.n:
            r_s = self.cShake_128(seed,self.c * self.beta/8,D_prime)
            D_prime += 1
            for j in range(0,self.c):
                number = int.from_bytes(r_s[j*self.beta/8:(j+1)*self.beta/8],"big")
                sign = number // (2**(self.beta - 1))
            
                val = self.remove_top_bits(number)
                for k in range(0,self.t):
                    if val >= self.get_cdt_v(k):
                        z[i+j] += 1
                
                if sign == 1:
                    z[i+j] = -z[i+j]
            
            i += self.c
        
        return z
    
    def remove_top_bits(self,byte):
        value = byte & 0x00000000FFFFFFFF
        print(byte)
        print(value)
        return value
    
    def get_cdt_v(self,k):
        print(self.cdt_v[2*k + 1])
        return self.cdt_v[2*k + 1 ] 
    
    def ySampler(self,seed,counter):
        self.b = math.ceil((math.log(self.B,2) + 1)/8)
        pos = 0
        i = 0
        y_s = []
        n_prime = self.n
        D_prime = counter * (2**8)

        c_s = self.cShake_128(seed,self.b * n_prime,D_prime)
        while i < self.n:
            if pos >= n_prime:
                D_prime += 1
                pos = 0
                n_prime = math.floor(self.rate_xof / self.b)
                c_s = self.cShake_128(seed,self.rate_xof,D_prime)
            converted_c = int.from_bytes(c_s[pos*self.b:(pos + 1)*self.b],byteorder="little")

            res = converted_c % (2**(math.ceil(math.log(self.B,2)) + 1)) - self.B
            if res != self.B+1:
                i += 1
                y_s.append(res)
            pos += 1
            
        return y_s
    
    def poly_add(self,poly_one,poly_two):
        res = [0] * self.n
        i = 0
        for entry in poly_one:
            res[i] = entry
            i+=1
            if i >= self.n:
                break
        
        i = 0
        for entry in poly_two:
            res[i]+= entry
            i+=1
            if i >= self.n:
                break
        
        return res
    
    def poly_sub(self,poly_one,poly_two):
        res = [0] * self.n
        i = 0
        for entry in poly_one:
            res[i] = entry
            i+=1
            if i >= self.n:
                break
        
        i = 0
        for entry in poly_two:
            res[i]-= entry
            i+=1
            if i >= self.n:
                break
        
        return res
            
    
    def poly_multiplication(self,poly_one,other_poly):
        result_poly = []
        poly_two = self.ntt(self.R(other_poly))
        for i in range(0,self.n):
            result_poly.append(poly_one[i] * poly_two[i])
        return self.ntt_inverse(result_poly)
    
    def sparse_multiplication(self,c,g_poly):
        (pos_list,sign_list) = c
        f = [0] * self.n
        for i in range(0,self.h):
            pos = pos_list[i]
            for j in range(0,pos):
                f[j] = f[j] - sign_list[i]*g_poly[j+self.n-pos]
            for j in range(pos,self.n):
                f[j] = f[j] + sign_list[i]*g_poly[j-pos]
        return f
    
    def PRF2(self,mess):
        return self.PRF1(mess)
    
    def PRF1(self,message):
        shake = hashlib.shake_128()
        shake.update(message)
        size = math.ceil((self.k_bits * (self.k + 3)) / 8)
        res = shake.digest(size)
        
        return (res[:self.k_bytes],res[self.k_bytes:self.k_bytes*(self.k + 1)],res[self.k_bytes*(self.k+1):self.k_bytes*(self.k+2)],res[self.k_bytes*(self.k+2):self.k_bytes*(self.k+3)])
    
    def shake_128(self,message,size):
        shake = hashlib.shake_128()
        shake.update(message)

        return shake.digest(int(size))
    
    def cShake_128(self,c_prime,rate,D=None):
        if D == None:
            return self.shake_128(c_prime,rate)
        else:
            return self.keccak(c_prime,D,rate)
    
    def keccak(self,c_prime,D,rate):
        shake = hashlib.shake_128()
        update_string = int(D).to_bytes(136,"big") + c_prime + b"\x00\x00"
        shake.update(update_string)
        return shake.digest(int(rate))
    
    def G(self,message):
        # The algorithm specifies 40 bytes
        # Convert array of polys somehow to bytes
        return self.shake_128(message,40)
    
    def H(self,v_polys,digest_one,digest_two):
        w = [0]*(self.k*self.n + 79)
        for i in range(0,self.k):
            for j in range(0,self.n):
                
                val = int(v_polys[i][j]) % (2**self.d)
                
                if val > (2**(self.d-1)):
                    val = val - (2**self.d)
                w[i*self.n + j] = (int(v_polys[i][j]) - val)/(2**self.d) + 128
        w[self.k*self.n:self.k*self.n + 40] = digest_one
        w[self.k*self.n + 40:] = digest_two

        c_prime = self.shake_128(bytes(w),self.k_bits/8)
        
        return c_prime
    
    def mod_minus_plus(self,poly,q=None):
        
        if q == None:
            q = self.q
        
        entries = []
        
        for entry in poly:
            res = int(entry) % q
            res -= math.floor(q/2)
            entries.append(res)
        
        return entries
        
    
    def order_quo(self,polynomial):
        # Order polynomial quoficientes
        quofficients = []
        for i in polynomial:
            quofficients.append(i)
        quofficients.sort(reverse=True)
        return quofficients
        
    
    def infinite_norm_check(self,polynomial):
        sum = 0
        ordered = self.order_quo(polynomial)
        for i in range(0,self.h):
            sum +=  ordered[i]
        return sum
    
    def checkS(self,s):
        return  self.infinite_norm_check(s) <= self.Ls
    
    def checkE(self,e):
        return  self.infinite_norm_check(e) <= self.Le
    
    def low(self,poly):
        res = self.mod_minus_plus(poly)
        return self.mod_minus_plus(res,2**self.d)
    
    def well_rounded(self,poly):
        
        flag1 = self.isShort(poly,math.floor(self.q/2) - self.E)
        flag2 = self.isShort(self.low(poly),2**(self.d-1) - self.E)
       
        
        print("FLAGS")
        print(flag1)
        print(flag2)

        
        return flag1 and flag2
    
    def isShort(self,poly,short):
        flag = True
        i = 0
        for entry in poly:
            if abs(int(entry)) > abs(short):
                flag = False
                break
            i+=1
                
        return flag
    
    def keyGen(self):
        t = []
        e_polys = []
        counter = 1
        pre_seed = os.urandom(self.k_bits/8)
        (seed_s,seed_es,seed_a,seed_y) = self.PRF1(pre_seed)

        a_polys = self.GenA(seed_a)
        

        s = self.GaussSampler(seed_s,counter)
        counter += 1
        print("Initialization")
        while not self.checkS(s):
            s = self.GaussSampler(seed_s,counter)
            counter += 1
        print("Secret Done")
        for i in range(0,self.k):
            e_i = self.GaussSampler(seed_es[i*self.k_bytes:(i+1)*self.k_bytes],counter)
            counter += 1

            while not self.checkE(e_i):
                
                e_i = self.GaussSampler(seed_es[i*self.k_bytes:(i+1)*self.k_bytes],counter)
                counter += 1
                
            e_polys.append(e_i)
            # mod q is missing, in next line
            t.append(self.poly_add(self.poly_multiplication(a_polys[i],s),e_i))
        print("Putting together public and secret key")
        g = self.G(pickle.dumps(t))
        sk = (s,e_polys,seed_a,seed_y,g)
        pk = (t,seed_a)
        print("Returnin")
        return (sk,pk)
    
    def sign(self,message,sk):
        counter = 1
        v_polys = []
        (s,e_polys,seed_a,seed_y,g) = sk
        
        r = os.urandom(self.k_bytes)
        rand = self.PRF2(seed_y + r + self.G(message))
        
        a_polys = self.GenA(seed_a)
        print("Initializion Done")
        while True:
            
            y = self.ySampler(rand[0] + rand[1] + rand[2] + rand[3],counter)

            for i in range(0,self.k):
                res = self.poly_multiplication(a_polys[i],y).list()
                v_polys.append(self.mod_minus_plus(res))

            c_prime = self.H(v_polys,self.G(message),g)
            c = self.Enc(c_prime)
            mult_result = self.sparse_multiplication(c,s)
        
            z = self.poly_add(mult_result,y)
        
            
            if self.isShort(z,self.B - self.S):
                well_rounded = True
                
                for i in range(0,self.k):
                    
                    res = self.sparse_multiplication(c,e_polys[i])
                    res = self.poly_sub(v_polys[i],res)
                    res = self.mod_minus_plus(res)
                    
                    if not self.well_rounded(res):
                        well_rounded = False
                        break
                        
                if well_rounded:
                    break
            
            
            counter+=1
        
        print("Returning")
        return (z,c_prime)
            
    def verify(self,message,sign,pk):
        (z,c_prime) = sign
        if not self.isShort(z,self.B - self.S):
            print("me fake")
            return False
        (t_polys,seed_a) = pk
        c = self.Enc(c_prime)
        a_polys = self.GenA(seed_a)
        w_polys = []
        for i in range(0,self.k):
            res1 = self.poly_multiplication(a_polys[i],z).list()
            res2 = self.sparse_multiplication(c,t_polys[i])
            w_polys.append(self.mod_minus_plus(self.poly_sub(res1,res2)))
            
        print(self.H(w_polys,self.G(message),self.G(pickle.dumps(t_polys))))
        if c_prime == self.H(w_polys,self.G(message),self.G(pickle.dumps(t_polys))):
            return True
        print("me not equal")
        return False
                           
                           
qTes = qTesla()

(sk,pk) = qTes.keyGen()

(z,c_prime) = qTes.sign(b"this is something",sk)
                           
print("signature:")
print(c_prime)

qTes.verify(b"this is something",(z,c_prime),pk)
                           
                           
                           
                           
                           
                           

6754599706910257071
1649930159
0
671507412
372236861
1131554536
80027618
352172656
1164298592
860014474
1378472045
1948467327
839957239
1125857607
2009293508
664324558
1466301668
506071440
234057451
671584905
786178128
306011771
714310105
243698855
417712145
9470578
1840927014
1246948843
589890969
1774056149
1109874008
1056451611
1812177762
829172876
313414831
1956430050
1160031633
1398244789
187242113
321666415
1304194029
2048797972
1282326805
831849416
1574767936
194943011
1991776993
2120655340
653713809
799217300
1380433609
1329670087
1873439229
103862387
254367675
1339200562
754636301
1499965744
1850514943
2013121736
2087512222
2121077103
2136013361
2142568585
2145405997
2146617281
2147127267
2147339035
2147425762
2147460791
2147474745
2147480227
2147482351
2147483163
2147483469
2147483583
2147483625
2147483640
2147483645
2147483647
10468333516895481182
3101617502
0
671507412
372236861
1131554536
80027618
352172656
1164298592
860014474
1378472045
1948467327
839957239
1125857607
200

KeyboardInterrupt: 